In [37]:
from text_preprocess import *
import pandas as pd
import os
import json
import csv

In [5]:
DATA_PATH = '../data/stumbleupon/'

In [16]:
train_df = pd.read_csv(os.path.join(DATA_PATH, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(DATA_PATH, 'test.tsv'), sep='\t')

In [8]:
train_df.head(5)

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,...,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label
0,http://www.bloomberg.com/news/2010-12-23/ibm-p...,4042,"{""title"":""IBM Sees Holographic Calls Air Breat...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,...,1,1,24,0,5424,170,8,0.152941,0.079130,0
1,http://www.popsci.com/technology/article/2012-...,8471,"{""title"":""The Fully Electronic Futuristic Star...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,...,1,1,40,0,4973,187,9,0.181818,0.125448,1
2,http://www.menshealth.com/health/flu-fighting-...,1164,"{""title"":""Fruits that Fight the Flu fruits tha...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,...,1,1,55,0,2240,258,11,0.166667,0.057613,1
3,http://www.dumblittleman.com/2007/12/10-foolpr...,6684,"{""title"":""10 Foolproof Tips for Better Sleep ""...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,...,1,0,24,0,2737,120,5,0.041667,0.100858,1
4,http://bleacherreport.com/articles/1205138-the...,9006,"{""title"":""The 50 Coolest Jerseys You Didn t Kn...",sports,0.719157,2.676471,0.500000,0.222222,0.123457,0.043210,...,1,1,14,0,12032,162,10,0.098765,0.082569,0


In [10]:
# .iloc parser
# just for fun
def boilerplate_parser_iloc(df):
    title = []
    text = []
    domain = []
    for i in range(len(df)):
        json_text = df["boilerplate"][i]
        s_dict = json.loads(json_text)
        title.append(s_dict.get('title', ''))
        text.append(s_dict.get('body', ''))
        domain.append(s_dict.get('url', ''))
    df['title'] = title
    df['text'] = text
    df['domain'] = domain
    return df

# apply parser
# for efficiency
def boilerplate_parser_apply(row):
    json_text = row['boilerplate']
    s_sict = json.loads(json_text)
    boilerplate_df = pd.Series({
        "title": s_sict.get('title', ''),
        "text": s_sict.get('body', ''),
        "domain": s_sict.get('url', '')
    })
    return boilerplate_df

In [11]:
boilerplate_df = train_df.apply(boilerplate_parser_apply, axis=1)

In [12]:
# Lets check if their is missing text data in dataframe
indexes = boilerplate_df[boilerplate_df['text'].isna()==True].index
len(indexes)

57

In [17]:
boilerplate_df_n = boilerplate_df.drop(index=indexes)
train_df = train_df.drop(index=indexes)

In [18]:
tokens = boilerplate_df_n['text'].apply(tokenizer)

In [40]:
type(tokens)

pandas.core.series.Series

In [20]:
vocabulary = [token for sublist in tokens for token in sublist]
word_to_idx = {word: idx for idx, word in enumerate(set(vocabulary))}
idx_to_word = {idx: word for idx, word in enumerate(set(vocabulary))}

In [55]:
decoded_tokens = tokens.apply(decode, word_to_idx=word_to_idx)

In [65]:
y = train_df['label']
y.to_csv(os.path.join(DATA_PATH, 'target.csv'), index=False)

In [64]:
df = pd.DataFrame(decoded_tokens)

df['text'] = df['text'].apply(json.dumps)

df.to_csv(os.path.join(DATA_PATH, 'decoded_tokens.csv'), index=False)